In [2]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
import optuna

sys.path.append('../../')

from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython


In [3]:
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


In [5]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

test_users = pd.read_csv('../data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [6]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [7]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [8]:
URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10])

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10])

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10])

URM_train_vector = [URM_train_1, URM_train_2, URM_train_3]
Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3]

In [9]:


class Objective(object):
    def __init__(self, URM_train_vector, ICM_all, Evaluator_vector):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train_vector = URM_train_vector
        self.ICM_all = ICM_all
        self.Evaluator_vector = Evaluator_vector
        self.MAP_vector = np.zeros(3)

        

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        topK = trial.suggest_int('topK', 50,800)
        weight = trial.suggest_uniform('weight', 0.1, 0.9)
        lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
        epochs= trial.suggest_int('epochs', 10,150)
        self.MAP_vector = np.zeros(3)
        i = 0
        for URM_train, evaluator_validation in zip(self.URM_train_vector,self.Evaluator_vector):
            URM_train_ICM_all = sps.vstack([URM_train*(1-weight), ICM_all.T*weight])
            recommender = SLIM_BPR_Cython(URM_train_ICM_all, recompile_cython=False)
            recommender.fit(epochs=epochs, sgd_mode= "adam", learning_rate=lr, topK = topK)
            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
            self.MAP_vector[i]=result_dict[10]["MAP"]
            i = i+1
        
        print('printing self map vector: ', self.MAP_vector)
        MAP = np.sum(self.MAP_vector)/3
        print('printing MAP: ', MAP)



        
        return MAP
    
# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(direction='maximize')
study.optimize(Objective(URM_train_vector, ICM_all, Evaluator_vector), n_trials=50)

print(study.best_params)

[I 2020-12-31 12:48:44,666] A new study created in memory with name: no-name-dd783966-3f7d-4a5b-9557-4a211e633414


SLIM_BPR_Recommender: URM Detected 68 (0.24 %) cold users.
SLIM_BPR_Recommender: URM Detected 1 (0.00 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 27947 ( 100.00% ) in 0.66 seconds. BPR loss is 1.69E-06. Sample per second: 42509
SLIM_BPR_Recommender: Epoch 1 of 200. Elapsed time 0.08 sec
Processed 27947 ( 100.00% ) in 0.74 seconds. BPR loss is 1.22E-05. Sample per second: 37537
SLIM_BPR_Recommender: Epoch 2 of 200. Elapsed time 0.17 sec
Processed 27947 ( 100.00% ) in 0.83 seconds. BPR loss is 2.60E-05. Sample per second: 33738
SLIM_BPR_Recommender: Epoch 3 of 200. Elapsed time 0.25 sec
Processed 27947 ( 100.00% ) in 0.91 seconds. BPR loss is 4.87E-05. Sample per second: 30643
SLIM_BPR_Recommender: Epoch 4 of 200. Elapsed time 0.33 sec
Processed 27947 ( 100.00% ) in 0.99 seconds. BPR loss i

[I 2020-12-31 12:52:56,100] Trial 0 finished with value: 0.056554505516261115 and parameters: {'topK': 577, 'weight': 0.6156320268490693, 'lr': 0.00010307360606886703}. Best is trial 0 with value: 0.056554505516261115.


printing self map vector:  [0.05619201 0.05538969 0.05808181]
printing MAP:  0.056554505516261115
Deallocating Cython objects
SLIM_BPR_Recommender: URM Detected 68 (0.24 %) cold users.
SLIM_BPR_Recommender: URM Detected 1 (0.00 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 27947 ( 100.00% ) in 0.99 seconds. BPR loss is 2.98E-07. Sample per second: 28091
SLIM_BPR_Recommender: Epoch 1 of 200. Elapsed time 0.08 sec
Processed 27947 ( 100.00% ) in 1.07 seconds. BPR loss is 1.89E-06. Sample per second: 26046
SLIM_BPR_Recommender: Epoch 2 of 200. Elapsed time 0.16 sec
Processed 27947 ( 100.00% ) in 0.15 seconds. BPR loss is 4.59E-06. Sample per second: 184922
SLIM_BPR_Recommender: Epoch 3 of 200. Elapsed time 0.23 sec
Processed 27947 ( 100.00% ) in 0.23 seconds. BPR loss is 8.36E-06. Sample per se

[I 2020-12-31 12:55:48,770] Trial 1 finished with value: 0.05503851747421079 and parameters: {'topK': 77, 'weight': 0.497321386674762, 'lr': 4.363837449009018e-05}. Best is trial 0 with value: 0.056554505516261115.


...

[I 2020-12-31 13:39:15,174] Trial 11 finished with value: 0.056732189497247114 and parameters: {'topK': 777, 'weight': 0.45897000787198283, 'lr': 0.0006957567271345782}. Best is trial 8 with value: 0.05716908969519335.


printing self map vector:  [0.05714368 0.05536935 0.05768353]
printing MAP:  0.056732189497247114
Deallocating Cython objects
SLIM_BPR_Recommender: URM Detected 68 (0.24 %) cold users.
SLIM_BPR_Recommender: URM Detected 1 (0.00 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 27947 ( 100.00% ) in 0.83 seconds. BPR loss is 6.19E-05. Sample per second: 33551
SLIM_BPR_Recommender: Epoch 1 of 200. Elapsed time 0.08 sec
Processed 27947 ( 100.00% ) in 0.91 seconds. BPR loss is 1.65E-04. Sample per second: 30675
SLIM_BPR_Recommender: Epoch 2 of 200. Elapsed time 0.16 sec
Processed 27947 ( 100.00% ) in 0.99 seconds. BPR loss is 4.68E-04. Sample per second: 28252
SLIM_BPR_Recommender: Epoch 3 of 200. Elapsed time 0.25 sec
Processed 27947 ( 100.00% ) in 0.08 seconds. BPR loss is 6.62E-04. Sample per sec

[I 2020-12-31 13:44:35,233] Trial 12 finished with value: 0.05713681876065133 and parameters: {'topK': 677, 'weight': 0.33928061156420664, 'lr': 0.00039177874296917784}. Best is trial 8 with value: 0.05716908969519335.


printing self map vector:  [0.05748608 0.05553314 0.05839124]
printing MAP:  0.05713681876065133
Deallocating Cython objects
SLIM_BPR_Recommender: URM Detected 68 (0.24 %) cold users.
SLIM_BPR_Recommender: URM Detected 1 (0.00 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 27947 ( 100.00% ) in 1.02 seconds. BPR loss is 6.65E-05. Sample per second: 27280
SLIM_BPR_Recommender: Epoch 1 of 200. Elapsed time 0.08 sec
Processed 27947 ( 100.00% ) in 0.10 seconds. BPR loss is 1.39E-04. Sample per second: 272421
SLIM_BPR_Recommender: Epoch 2 of 200. Elapsed time 0.16 sec
Processed 27947 ( 100.00% ) in 0.20 seconds. BPR loss is 2.97E-04. Sample per second: 142349
SLIM_BPR_Recommender: Epoch 3 of 200. Elapsed time 0.25 sec
Processed 27947 ( 100.00% ) in 0.27 seconds. BPR loss is 5.93E-04. Sample per se

KeyboardInterrupt: 